In [263]:
# Function to replace characters in coures prerequisite based on pattern
def parse_prerequisite(rep, text):
    # Replacement
    rep = dict((re.escape(k), v) for k, v in rep.iteritems())
    pattern = re.compile("|".join(rep.keys()))
    text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
    return text

In [356]:
# Function to split prerequisite and corequisite from coures prerequisites section
def split_prerequisite(text):
    # Has prerequisite(s)
    if len(text) > 1: 
        text_2 = text[1].split('.')
        
        # Switch course number to unicode
        uniString = text_2[0].encode('utf-8') 
        
        # Replace non-breaking space with space
        prerequisite_uniString = uniString.replace("\xc2\xa0", "") 
        
        # define desired first level replacements
        rep = {" or X": ";",
               ", ": "; ", 
               " and ": "; ",
               " / ": "; "} 
        
        parsed = parse_prerequisite(rep, prerequisite_uniString)
        
        # define desired second level replacements
        rep = {" or ": "; "}
        parsed = parse_prerequisite(rep, parsed)
        
        # define desired third level replacements
        rep = {";;": ";"}
        parsed = parse_prerequisite(rep, parsed)
        
        # Skip if string "units" in the prerequisite or corequisite
        if re.search('units', parsed) is None:   
            regex = re.compile('[A-Z]*\s?\d{1,}\w?')
        
            # Change prerequisite course list to ';' delimited string
            parsed_delimited = ';'.join(map(str, regex.findall(parsed.strip())))
        else:
            parsed_delimited = ""
        
        return parsed_delimited 

In [122]:
# define desired replacements
rep = {" or X": ";",
       ", ": "; ", 
       " and ": "; ",
      " / ": "; "} 

prerequisite_uniString = '  MUSIC 256A / CS 476A'
parsed = parse_prerequisite(rep, prerequisite_uniString)
print parsed
        
# define desired replacements
rep = {" or ": "; "}
parsed = parse_prerequisite(rep, parsed)

print parsed

# define desired replacements
rep = {";;": ";"}
parsed = parse_prerequisite(rep, parsed).strip()

print parsed
print "test"

  MUSIC 256A; CS 476A
  MUSIC 256A; CS 476A
MUSIC 256A; CS 476A
test


In [357]:
from bs4 import BeautifulSoup; #import beautiful soup scraper
import urllib2 #url library
import pandas as pd; #dataframe for final output
import re #regular expression package
from datetime import date

#%%
#url= 'http://www.pugetsound.edu/academics/faculty-scholarship/faculty-list/all/' #set url

# Set url
url_1 = 'http://exploredegrees.stanford.edu/schoolofengineering/computerscience/#courseinventory'
url_2 = 'http://exploredegrees.stanford.edu/schoolofengineering/electricalengineering/#courseinventory'

page_1 = urllib2.urlopen(url_1) #open the url
soup_1 = BeautifulSoup(page_1.read(), "lxml") #read in beautiful soup

page_2 = urllib2.urlopen(url_2) #open the url
soup_2 = BeautifulSoup(page_2.read(), "lxml") #read in beautiful soup

course_block_title_1 = [s.get_text() for s in soup_1.find_all("p", {"class":"courseblocktitle"}, "strong")] # Get course black title
course_number_title_1 = course_block_title_1

course_block_title_2 = [s.get_text() for s in soup_2.find_all("p", {"class":"courseblocktitle"}, "strong")] # Get course black title
course_number_title_2 = course_block_title_2

course_block_desc_1 = [s.get_text() for s in soup_1.find_all("p", {"class":"courseblockdesc"})] # Get course requsites
course_block_desc_2 = [s.get_text() for s in soup_2.find_all("p", {"class":"courseblockdesc"})] # Get course requsites

# Create empty lists for each course attribute
courseNumber = [] 
courseTitle = []
prerequisite = []
dateScrape = []
regex = []
    
# Append course number and couse title from courses in Computer Science Department
for each in course_number_title_1:
    course = each.split('.') # Split at period
    
    # Switch course number to unicode
    uniString = course[0].encode('utf-8') 
    
    # Replace non-breaking space with space
    course_uniString = uniString.replace("\xc2\xa0", "") 
    
    # Append to the course number list
    courseNumber.append(course_uniString.strip()) 
    
    # Switch course title to unicode and append to course title list
    courseTitle.append(course[1].encode('utf-8').strip()) 

# Append course number and couse title from courses in Electrical Engineering Department
for each in course_number_title_2:
    course = each.split('.') # Split at period
    
    # Switch course number to unicode
    uniString = course[0].encode('utf-8') 
    
    # Replace non-breaking space with space
    course_uniString = uniString.replace("\xc2\xa0", "") 
    
    # Append to the course number list
    courseNumber.append(course_uniString.strip()) 
    
    # Switch course title to unicode and append to course title list
    courseTitle.append(course[1].encode('utf-8').strip()) 

# Append course description from courses in Computer Science Department
for each in course_block_desc_1:
    # Parse prerequisite using delimiter ':'
    split_1 = re.split(r'Prerequisite[s]?[:]\s*', each)
    
    # Parse corequisite using delimiter ':'
    coreq_split_1 = re.split(r'Corequisite[s]?[:]\s*', each)
    
    # Append the prerequisite and corequisite course list to the course
    if (split_prerequisite(split_1) is not None) and \
        (split_prerequisite(coreq_split_1) is not None):
        prerequisite.append(split_prerequisite(split_1) + \
                            split_prerequisite(coreq_split_1)) 
        
    elif (split_prerequisite(split_1) is not None) and \
          (split_prerequisite(coreq_split_1) is None):
        prerequisite.append(split_prerequisite(split_1)) 
          
    elif (split_prerequisite(split_1) is None) and \
          (split_prerequisite(coreq_split_1) is not None):
        prerequisite.append(split_prerequisite(coreq_split_1))
          
    else:
        prerequisite.append(split_prerequisite(coreq_split_1)) # Can also use split_1

# Append course number and couse title from courses in Electrical Engineering Department
for each in course_block_desc_2:
    # Parse prerequisite using delimiter ':'
    split_1 = re.split(r'Prerequisite[s]?[:]\s*', each)
    
    # Parse corequisite using delimiter ':'
    coreq_split_1 = re.split(r'Corequisite[s]?[:]\s*', each)
    
    # Append the prerequisite and corequisite course list to the course
    if (split_prerequisite(split_1) is not None) and \
        (split_prerequisite(coreq_split_1) is not None):
        prerequisite.append(split_prerequisite(split_1) + \
                            split_prerequisite(coreq_split_1)) 
        
    elif (split_prerequisite(split_1) is not None) and \
          (split_prerequisite(coreq_split_1) is None):
        prerequisite.append(split_prerequisite(split_1)) 
          
    elif (split_prerequisite(split_1) is None) and \
          (split_prerequisite(coreq_split_1) is not None):
        prerequisite.append(split_prerequisite(coreq_split_1))
          
    else:
        prerequisite.append(split_prerequisite(coreq_split_1)) # Can also use split_1

    
#     # Parse prerequisite using delimiter ':'
#     split_1 = re.split(r'Prerequisite[s]?[:]\s*', each)
    
#     # Append the prerequisite course list to the course
#     prerequisite.append(split_prerequisite(split_1))
    
#     # Parse corequisite using delimiter ':'
#     coreq_split_1 = re.split(r'Corequisite[s]?[:]\s*', each)
    
#     # Append the corequisite course list to the course
#     prerequisite.append(split_prerequisite(coreq_split_1))
      
# Create a list of date for the date of scrape
dateScrape = [date.today().strftime('%m/%d/%Y')] * len(courseNumber)

# Set department
department_1 = ['Computer Science'] * len(course_number_title_1)
department_2 = ['Electrical Engineering'] * len(course_number_title_2)

# Set university
university = "Stanford"

print "prerequisite: ", prerequisite
print "length of prerequisite: ", len(prerequisite)
print "length of prerequisite: ", len(dateScrape)

print "Add all data to a dataframe"
output = pd.DataFrame(dateScrape) #create a dataframe for outputs
output.columns = ['Date of Scraping']
output['University'] = university
output['Department'] = department_1 + department_2
output['Course Number'] = courseNumber
output['Course Title'] = courseTitle
output['Prerequisite'] = prerequisite

# Output data to a csv file
print "Output data to file"
output.to_csv('wcao-INFX575-PS1_dataset.csv', index=False) # Skip index column

prerequisite:  [None, None, None, 'CS106B', None, None, None, None, 'CS107', None, None, None, None, None, None, '106B', None, '106A', None, 'CS106A', '106A', '', '', None, '106A', '106B; 106X', ' 106A', '106B', '106B', '107', '103; 106B;MATH51;CME100', 'CS109', '107', 'CS 106', 'CS103;CS107;CS109', '', 'CS110', 'CS107;CS108', '103; 103B; 107', 'CS110', '103; 107', '106B', 'CS107;MATH51', 'CS103; 103B', '110', '103; 103B', '103; 103B; 109;STATS116', 'CS107;CS161', 'CS161', 'CS107;CS161', None, 'CS107', '106B', None, None, '', 'CS140', '', '', None, '106B; 106B', ' 106A', ' 106B', 'CS193P', 'CS110;CS161', 'CS147', None, '1C', None, 'CS198', None, None, None, '', '106B;MATH51', '205A', 'CS109; 110', 'CS210A', None, 'CS107', 'CS103;CS 103B;CS106B;CS106X;CS107;CS109', None, '', 'CS124;CS121;221', 'CS124; 221; 224N; 229', 'LINGUIST180;CS124;CS224N;CS224S;CS221;LINGUIST130A;CS157;PHIL150', None, '223A', ' 106B; 106X; 205', '', '', '', 'STATS315A;CS229', '', 'CS2223B', 'CS131;CS229;MATH21', N

In [ ]:
(?!units)[A-Z]*\s?\d{1,}\w?\b(?=\)| \(|;|$)

In [302]:
a = re.compile("(?!units)[A-Z]*\s?\d{1,}\w?\b(?=\)| \(|;|$)", flags=re.IGNORECASE)
text = "Extracting meaning, information, and structure from human language text, \
speech, web pages, genome sequences, social networks. Methods include: string \
algorithms, edit distance, language modeling, the noisy channel, naive Bayes, \
inverted indices, collaborative filtering, PageRank. Applications such as question\
answering, sentiment analysis, information retrieval, text classification, social\
network models, machine translation, genomic sequence alignment, spell checking,\
speech processing, recommender systems. Prerequisite: 135 units."
split_1 = text.split('Prerequisite:')
a.split(split_1[1])

[' 135 units.']

In [355]:
# p = re.compile('\d+')
# p.findall('12 drummers drumming, 11 pipers piping, 10 lords a-leaping')
text = "Basic knowledge of computer programming (as covered in CS 106)"

regex = re.compile('[A-Z]*\s?\d{1,}\s?\w?')
regex.findall(text)

['CS 106']

In [202]:
myList = ['CS103', 'CS107', 'CS109']
myList = ';'.join(map(str, myList)) 
myList

'CS103;CS107;CS109'

In [274]:
a="Prerequisit: consent of instructor. Corequisite: 106B or 106X."
b1 = re.split(r'Corequisite[s]?[:]\s*', a)
b2 = re.split(r'Prerequisite[s]?[:]\s*', a)
print b[0]
print b[1]

if split_prerequisite(b2) is not None:
    split_prerequisite(b1)+split_prerequisite(b2)

Prerequisite: consent of instructor. 
106B or 106X.
